In [ ]:
# =========================
# Imports
# =========================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import librosa
import librosa.display
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.utils.class_weight import compute_class_weight

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers, callbacks
from tensorflow.keras.utils import to_categorical

from google.colab import drive

import os, numpy as np
from sklearn.model_selection import GroupShuffleSplit

In [ ]:
# =========================
# Montage Drive + chemins
# =========================
drive.mount("/content/drive")
BASE = "/content/drive/MyDrive/IA/bird_songs_dataset"
CSV  = "/content/drive/MyDrive/IA/bird_songs_metadata.csv"


In [ ]:
# =========================
# Chargement CDV
# =========================
label_df = pd.read_csv(CSV)
print("Aperçu des colonnes:", list(label_df.columns))

if "filepath" not in label_df.columns:
    if "filename" in label_df.columns:
        label_df["filepath"] = label_df["filename"].apply(lambda fn: os.path.join(BASE, str(fn)))
    else:
        raise ValueError("Aucune colonne 'filename' ou 'filepath' trouvée dans le CSV.")

label_df["exists"] = label_df["filepath"].apply(os.path.exists)
missing = label_df[~label_df["exists"]]
print("Nb de fichiers manquants:", len(missing))
if len(missing):
    print("Exemples de fichiers manquants :")
    print(missing.head())
    label_df = label_df[label_df["exists"]].copy()

print("Nb de lignes après filtrage:", len(label_df))

### Définitions :

**Amplitude** :
Valeur instantanée du signal audio (proportionnelle à la pression acoustique). Elle détermine l’énergie/le volume perçu.

**Fréquence** :
Nombre d’oscillations par seconde (Hz). Les oiseaux chantent dans des bandes de fréquences caractéristiques.

**Nombre de canaux** :
Mono (1 canal) ou stéréo (2 canaux). Permet une spacialisation du son (2 canaux et +).

**Fréquence d’échantillonnage (SR)** :
Nombre d’échantillons par seconde (Hz). Généralement plus elle est haute plus la qualité est bonne.

### Format WAV et en-têtes

Un fichier **WAV** est découpé en plusieurs blocs :

- RIFF : identifiant + taille totale du fichier.
- WAVE : signature indiquant le type de données audio.
- fmt : informations de format (fréquence d’échantillonnage, nombre de canaux, profondeur en bits).
- data : bloc contenant les échantillons audio bruts.

Exemple : un WAV 44,1 kHz / 16 bits / 2 canaux aura dans son header :
- Sample rate = 44100 Hz
- Bits per sample = 16
- Channels = 2
- Byte rate = 44100 × 2 (stéréo) × 16/8 = 176,4 kB/s

In [ ]:
# =========================
# Exploration basique + enrichissement CSV
# =========================
durations, orig_srs, n_channels = [], [], []
rms_vals, centroids, bandwidths = [], [], []

for filepath in tqdm(label_df["filepath"], desc="Scan audio (durée/sr/canaux + rms/ctr/bdw)"):
    y, sr = librosa.load(filepath, sr=None, mono=False)

    # canaux / durée
    if y.ndim == 1:
        channels = 1
        duration = len(y) / sr
    else:
        channels = y.shape[0]
        duration = y.shape[1] / sr

    # passage en mono pour features spectrales
    y_mono = y if y.ndim == 1 else np.mean(y, axis=0)

    # features rms - ctr - bdw
    try:
        rms = float(librosa.feature.rms(y=y_mono).mean())
    except Exception:
        rms = np.nan
    try:
        centroid = float(librosa.feature.spectral_centroid(y=y_mono, sr=sr).mean())
    except Exception:
        centroid = np.nan
    try:
        bw = float(librosa.feature.spectral_bandwidth(y=y_mono, sr=sr).mean())
    except Exception:
        bw = np.nan

    durations.append(duration)
    orig_srs.append(sr)
    n_channels.append(channels)
    rms_vals.append(rms)
    centroids.append(centroid)
    bandwidths.append(bw)

# Ajout colonnes enrichies
label_df["duration_s"]         = durations
label_df["orig_sr"]            = orig_srs
label_df["n_channels"]         = n_channels
label_df["rms_energy"]         = rms_vals
label_df["spectral_centroid"]  = centroids
label_df["spectral_bandwidth"] = bandwidths

print(label_df.head())

* **duration_s** = durée totale du fichier audio.

* **orig_sr** = fréquence d’échantillonnage d’origine.

* **n_channels** = nombre de canaux.

* **rms_energy** = : énergie racine carrée moyenne du signal. Elle reflète l’intensité/volume global d’un son. Valeurs hautes = son fort, basses = son faible.

* **spectral_centroid** = barycentre du spectre de fréquences. Souvent perçu comme une mesure de la “brillance” d’un son (plus le centroid est haut, plus le son paraît aigu/clair).

* **spectral_bandwidth** = largeur effective du spectre autour du centroid. Indique la dispersion des fréquences (sons purs = bande étroite, sons riches/bruités = bande large).

In [ ]:
# =========================
# Visualisation simple
# =========================

# 1) Un exemple d'onde par classe
classes = label_df["species"].unique()
plt.figure(figsize=(12, max(2.5, 2.5 * len(classes))))
for idx, cls in enumerate(classes):
    row = label_df[label_df["species"] == cls].sample(1, random_state=42).iloc[0]
    filepath = row["filepath"]
    y, sr = librosa.load(filepath, sr=22050, mono=True)
    t = np.arange(len(y)) / sr

    plt.subplot(len(classes), 1, idx + 1)
    plt.plot(t, y)
    plt.title(f"{cls} — {os.path.basename(filepath)}", fontsize=10)
    plt.xlabel("Temps (s)")
    plt.ylabel("Amplitude")
plt.tight_layout()
plt.show()

# 2) Répartition des classes
class_counts = label_df["species"].value_counts().sort_index()
print("\nNombre de sons par classe:\n", class_counts)
plt.figure(figsize=(8, 5))
class_counts.plot(kind="bar", edgecolor="black")
plt.title("Répartition des sons par classe")
plt.xlabel("Classe d'oiseau")
plt.ylabel("Nombre de fichiers")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# 3) Distribution du nombre de canaux
plt.figure(figsize=(4,3))
label_df["n_channels"].value_counts().plot.bar(color="skyblue")
plt.title("Distribution du nombre de canaux")
plt.xlabel("Nb canaux")
plt.ylabel("Comptage")
plt.show()

# 4) Distribution de la fréquence d'échantillonnage d'origine
plt.figure(figsize=(6,4))
sns.histplot(label_df["orig_sr"], bins=20, kde=True, color="orange")
plt.title("Distribution des fréquences d'échantillonnage originales")
plt.xlabel("Fréquence (Hz)")
plt.ylabel("Comptage")
plt.show()


# 5) Distribution durées
plt.figure(figsize=(8, 5))
sns.histplot(label_df["duration_s"], bins=50, kde=True)
plt.title("Distribution des durées des enregistrements")
plt.xlabel("Durée (secondes)")
plt.ylabel("Nombre de fichiers")
plt.show()

# Sauvegarde CSV enrichi
OUT = "/content/drive/MyDrive/IA/bird_songs_metadata_enriched.csv"
label_df.drop(columns=["exists"], errors="ignore").to_csv(OUT, index=False)
print(f"CSV enrichi sauvegardé : {OUT}")

# Pourquoi le tracé temps-amplitude ne suffit pas ?

La forme d’onde montre quand le signal est fort/faible, mais presque rien sur où se situe l’énergie en fréquence ou encore le timbre. Deux chants peuvent avoir des amplitudes similaires et des contenus fréquentiels très différents. C'est pour cela que nous allons utiliser le Mel-spectrogram, logmel spectrograme ou encore le MFCC pour récupérer des informations plus adaptées à la classification.

> Ajouter une citation



* Tout les enregistrements font 22050 Hz
* Tout les enregistrements font 3 secondes

L’écart est modéré (minimum à 890 et maximum à 1 250, écart max à 360).

-> Pas obligatoire d’équilibrer, on surveillera les metrics pour confirmer.

In [ ]:
# =========================
# Encodage des labels
# =========================
le = LabelEncoder()
y_all = le.fit_transform(label_df["species"].values)
y_all_ref = y_all.copy()
print("Classes:", le.classes_)
print("y_all shape:", y_all_ref.shape)

In [ ]:
# =========================
# Paramètres audio globaux
# =========================
SR      = 22050
N_FFT   = 2048
HOP     = 512
N_MFCC  = 40
N_MELS  = 128

TARGET_DUR = 3.0

# Définitions :

**Mel-spectrogram** : on calcule un spectrogramme par STFT  puis on projette l’énergie sur une banque de filtres Mel (représentation quasi-log de la hauteur perçue). On prend souvent le log de l’énergie (log-Mel) pour rapprocher la dynamique de l’oreille humaine.

**MFCC** : on applique une DCT (transformée en cosinus) sur le log-Mel pour décorréler les bandes et compacter l’info dans les premiers N coefficients (ici 40). Les MFCCs résument le timbre ; on peut ajouter Δ et ΔΔ (dérivées 1ère/2ème) pour capturer la dynamique temporelle.


In [ ]:
# =========================
# Utilitaires audio/features
# =========================
def load_fixed_mono(path, sr=SR, target_dur=TARGET_DUR):
    y, _ = librosa.load(path, sr=sr, mono=True)
    target_len = int(sr * target_dur)
    if len(y) < target_len:
        pad = target_len - len(y)
        y = np.pad(y, (0, pad), mode="constant")
    else:
        y = y[:target_len]
    return y

def mfcc_flat_from_path(path):
    """MFCC 2D -> vecteur 1D (pour MLP)."""
    y = load_fixed_mono(path)
    mf = librosa.feature.mfcc(y=y, sr=SR, n_mfcc=N_MFCC, n_fft=N_FFT, hop_length=HOP)
    mf = (mf - mf.mean(axis=1, keepdims=True)) / (mf.std(axis=1, keepdims=True) + 1e-8) # Si mf = matrice (40, 200), alors mf = (40, 1).
    return mf.flatten().astype(np.float32)

def mfcc_2d_from_path(path):
    """MFCC 2D normalisé, shape (N_MFCC, T, 1)."""
    y = load_fixed_mono(path)
    mf = librosa.feature.mfcc(y=y, sr=SR, n_mfcc=N_MFCC, n_fft=N_FFT, hop_length=HOP).astype(np.float32)
    mf = (mf - mf.mean(axis=1, keepdims=True)) / (mf.std(axis=1, keepdims=True) + 1e-8)
    return mf[..., np.newaxis]

def logmel_2d(y, sr=SR, n_fft=N_FFT, hop=HOP, n_mels=N_MELS, norm="per_band"):
    """Calcule un log-Mel 2D (N_MELS x T), normalisé."""
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop, n_mels=n_mels)
    logmel = librosa.power_to_db(mel, ref=np.max)

    if norm == "per_band":
        mu  = logmel.mean(axis=1, keepdims=True)
        std = logmel.std(axis=1, keepdims=True) + 1e-8
        logmel = (logmel - mu) / std
    elif norm == "global":
        logmel = (logmel - logmel.mean()) / (logmel.std() + 1e-8)

    return logmel.astype(np.float32)

In [ ]:
# ============================================
# Onde, STFT (dB), Mel, log-Mel, MFCC
# ============================================

row = label_df.sample(1, random_state=42).iloc[0]
fp = row["filepath"]
species = row.get("species", "unknown")
print("Fichier choisi:", fp, "| espèce:", species)

y = load_fixed_mono(fp, sr=SR, target_dur=TARGET_DUR)

# Onde temporelle
plt.figure(figsize=(10, 3))
t = np.arange(len(y)) / SR
plt.plot(t, y)
plt.title(f"Onde temporelle — {species}")
plt.xlabel("Temps (s)")
plt.ylabel("Amplitude")
plt.tight_layout(); plt.show()

# Spectrogramme STFT
S = np.abs(librosa.stft(y, n_fft=N_FFT, hop_length=HOP))**2
S_db = librosa.power_to_db(S, ref=np.max)

plt.figure(figsize=(10, 4))
librosa.display.specshow(S_db, sr=SR, hop_length=HOP, x_axis="time", y_axis="hz")
plt.title(f"Spectrogramme STFT (dB) — {species}")
plt.colorbar(format="%+0.1f dB")
plt.tight_layout(); plt.show()

# Mel-spectrogram
M = librosa.feature.melspectrogram(y=y, sr=SR, n_fft=N_FFT, hop_length=HOP, n_mels=N_MELS)

plt.figure(figsize=(10, 4))
librosa.display.specshow(M, sr=SR, hop_length=HOP, x_axis="time", y_axis="mel")
plt.title(f"Mel-spectrogram (puissance) — {species}")
plt.colorbar()
plt.tight_layout(); plt.show()

# log-Mel
M_db = librosa.power_to_db(M, ref=np.max)

plt.figure(figsize=(10, 4))
librosa.display.specshow(M_db, sr=SR, hop_length=HOP, x_axis="time", y_axis="mel")
plt.title(f"log-Mel (dB) — {species}")
plt.colorbar(format="%+0.1f dB")
plt.tight_layout(); plt.show()

# MFCC
mfcc = librosa.feature.mfcc(S=M_db, n_mfcc=N_MFCC)

plt.figure(figsize=(10, 4))
librosa.display.specshow(mfcc, sr=SR, hop_length=HOP, x_axis="time")
plt.title(f"MFCC (N={N_MFCC}) — {species}")
plt.colorbar()
plt.tight_layout(); plt.show()

In [ ]:
# =========================
# Helpers plots
# =========================
def plot_history(history, title_prefix=""):
    plt.figure()
    plt.plot(history.history["accuracy"])
    plt.plot(history.history["val_accuracy"])
    plt.title(f"{title_prefix} Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Acc")
    plt.legend(["train", "val"])
    plt.tight_layout()
    plt.show()

    plt.figure()
    plt.plot(history.history["loss"])
    plt.plot(history.history["val_loss"])
    plt.title(f"{title_prefix} Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(["train", "val"])
    plt.tight_layout()
    plt.show()

def plot_confusion(y_true, y_pred, labels=None, title="Matrice de confusion (normalisée)"):
    cm = confusion_matrix(y_true, y_pred, normalize="true")
    fig, ax = plt.subplots(figsize=(5, 4))
    im = ax.imshow(cm, interpolation="nearest", cmap="Blues")
    ax.figure.colorbar(im, ax=ax)
    if labels is None:
        labels = np.arange(len(np.unique(y_true)))
    ax.set_xticks(np.arange(len(labels))); ax.set_yticks(np.arange(len(labels)))
    ax.set_xticklabels(labels, rotation=45, ha="right"); ax.set_yticklabels(labels)
    ax.set_xlabel("Prédit"); ax.set_ylabel("Vrai")
    ax.set_title(title)
    plt.tight_layout()
    plt.show()

Le jeu de données contient plusieurs extraits issus d’un même enregistrement source, exemple :
* 557838-0.wav   
* 557838-1.wav   
* 557838-4.wav   

Si on fait un split aléatoir, des extraits quasi identiques (même oiseau, même lieu, même bruit de fond, même micro) peuvent se retrouver à la fois dans l’entraînement et dans la validation/test. Le modèle “reconnaît” alors des signatures parasites plutôt que des caractéristiques vraiment discriminantes de l’espèce ce qui provoque des scores artificiellement gonflés.


On impose donc un split par groupe d’enregistrement : on dérive un record_id à partir du nom de fichier afin de regrouper tous les extraits du même enregistrement.

Puis on utilise GroupShuffleSplit pour séparer train / val / test en veillant à ce qu’aucun record_id ne soit partagé entre les splits. Ainsi, un enregistrement complet n’apparaît que dans un seul ensemble, ce qui supprime la fuite d’information.

In [ ]:
# ============================================================
# Anti Leak
# ============================================================

if "filename" not in label_df.columns:
    label_df["filename"] = label_df["filepath"].apply(lambda p: os.path.basename(p))
def _recid(fn: str) -> str:
    base = os.path.splitext(fn)[0]
    return base.split("-")[0].split("_")[0]
label_df["record_id"] = label_df["filename"].apply(_recid)

files_all = label_df["filepath"].values
y_all     = np.asarray(y_all_ref, dtype=int)
groups    = label_df["record_id"].values

gss1 = GroupShuffleSplit(test_size=0.30, n_splits=1, random_state=42)
idx_train, idx_temp = next(gss1.split(files_all, y_all, groups=groups))
gss2 = GroupShuffleSplit(test_size=0.50, n_splits=1, random_state=42)
idx_val_rel, idx_test_rel = next(gss2.split(files_all[idx_temp], y_all[idx_temp], groups=groups[idx_temp]))
idx_val_abs  = idx_temp[idx_val_rel]
idx_test_abs = idx_temp[idx_test_rel]

def slice_split(X, y):
    return X[idx_train], X[idx_val_abs], X[idx_test_abs], y[idx_train], y[idx_val_abs], y[idx_test_abs]


Réseau de neurones pleinement connecté (MLP)

Le premier modèle implémenté est un réseau dense appliqué sur les MFCC vectorisés.
Bien que ce modèle capture une certaine structure, il ne prend pas en compte les relations temporelles et fréquentielles.
Le score obtenu est d’environ 46 % d’accuracy test (macro-F1 ≈ 0.45), ce qui montre une limite importante dans sa capacité à généraliser.

In [ ]:
# ============================================================
# PIPELINE 1 — MLP sur MFCC flatten (entrée 1D)
# ============================================================

# Extraction MFCC 1D
X_list, y_mlp_list = [], []

for fp, yi in tqdm(zip(label_df["filepath"].values, y_all), total=len(label_df), desc="MFCC flatten (MLP)"):
    try:
        X_list.append(mfcc_flat_from_path(fp))   # -> vecteur 1D
        y_mlp_list.append(yi)
    except Exception as e:
        print(f"Skip {fp} -> {e}")

# passage en matrice
X_mlp = np.vstack(X_list)
y_mlp = np.array(y_mlp_list, dtype=int)
print("MLP — Features:", X_mlp.shape, "| Labels:", y_mlp.shape)


df_mfcc = pd.DataFrame(X_mlp, columns=[f"mfcc_{i}" for i in range(X_mlp.shape[1])])
df_mfcc["species"] = [le.inverse_transform([y])[0] for y in y_mlp]

display(df_mfcc.head())
print("Shape du DataFrame MFCC+label :", df_mfcc.shape)

X_train, X_val, X_test, y_train, y_val, y_test = slice_split(X_mlp, y_mlp)

# Encodage (one-hot encoded)
num_classes_mlp = len(np.unique(y_mlp))
y_train_oh = to_categorical(y_train, num_classes_mlp)
y_val_oh   = to_categorical(y_val,   num_classes_mlp)
y_test_oh  = to_categorical(y_test,  num_classes_mlp)

model_mlp = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(num_classes_mlp, activation="softmax"),
])
model_mlp.compile(optimizer=keras.optimizers.Adam(1e-3),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
cbs_mlp = [
    callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(patience=3, factor=0.5),
]
history_mlp = model_mlp.fit(
    X_train, y_train_oh,
    validation_data=(X_val, y_val_oh),
    epochs=40, batch_size=64, callbacks=cbs_mlp, verbose=1
)

# Scores
y_pred_test_mlp = model_mlp.predict(X_test, batch_size=64).argmax(axis=1)
print("\n[MLP] Test accuracy:", (y_pred_test_mlp == y_test).mean())
print("[MLP] Macro-F1:", f1_score(y_test, y_pred_test_mlp, average="macro"))
print(classification_report(y_test, y_pred_test_mlp, target_names=le.classes_))

plot_history(history_mlp, title_prefix="[MLP]")
plot_confusion(y_test, y_pred_test_mlp, labels=le.classes_, title="[MLP] Matrice de confusion (normalisée)")

model_mlp.save("/content/drive/MyDrive/IA/Model Save/Model_MFCC_1D.keras")


Réseau de neurones convolutif (CNN, MFCC 2D)

Le deuxième modèle, un CNN appliqué sur les MFCC 2D, exploite directement la structure temps-fréquence du signal.
Cette approche permet d’apprendre des motifs locaux caractéristiques des espèces d’oiseaux.
Le modèle atteint environ 74 % d’accuracy test et un macro-F1 ≈ 0.74, marquant un progrès significatif par rapport au MLP.
L’analyse de la matrice de confusion montre une bonne reconnaissance de classes comme cardinalis (rappel = 0.93), mais certaines confusions persistent, notamment entre polyglottos et migratorius.

In [ ]:
# ============================================================
# PIPELINE 2 — CNN sur MFCC 2D
# ============================================================
X2_list, y_mfcc2d_list = [], []
for fp, yi in tqdm(zip(label_df["filepath"].values, y_all_ref),
                   total=len(label_df), desc="MFCC 2D (CNN)"):
    try:
        X2_list.append(mfcc_2d_from_path(fp))
        y_mfcc2d_list.append(yi)
    except Exception as e:
        print(f"Skip {fp} -> {e}")

X_mfcc2d = np.stack(X2_list)  # (N, N_MFCC, T, 1)
y_mfcc2d = np.array(y_mfcc2d_list, dtype=int)
print("MFCC2D — Features:", X_mfcc2d.shape, "| Labels:", y_mfcc2d.shape)

X2_train, X2_val, X2_test, y2_train, y2_val, y2_test = slice_split(X_mfcc2d, y_mfcc2d)

classes_unique = np.unique(y2_train)
cw = compute_class_weight(class_weight="balanced", classes=classes_unique, y=y2_train)
class_weight = {int(c): float(w) for c, w in zip(classes_unique, cw)}
print("class_weight:", class_weight)

num_classes_cnn_mfcc = len(np.unique(y_mfcc2d))

inp_mfcc = layers.Input(shape=(X2_train.shape[1], X2_train.shape[2], 1))
x = layers.Conv2D(32, (3,3), padding="same", activation="relu")(inp_mfcc)
x = layers.BatchNormalization()(x); x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(64, (3,3), padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x); x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(128, (3,3), padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x); x = layers.MaxPooling2D((2,2))(x)
x = layers.Dropout(0.4)(x); x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x); x = layers.Dropout(0.4)(x)
out_mfcc = layers.Dense(num_classes_cnn_mfcc, activation="softmax")(x)

model_cnn_mfcc = keras.Model(inp_mfcc, out_mfcc)
model_cnn_mfcc.compile(optimizer=keras.optimizers.Adam(1e-3),
                       loss="sparse_categorical_crossentropy",
                       metrics=["accuracy"])

cbs_cnn_mfcc = [
    callbacks.EarlyStopping(patience=6, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(patience=3, factor=0.5),
]

history_cnn_mfcc = model_cnn_mfcc.fit(
    X2_train, y2_train,
    validation_data=(X2_val, y2_val),
    epochs=40, batch_size=64, callbacks=cbs_cnn_mfcc, verbose=1,
    class_weight=class_weight
)

y2_pred = model_cnn_mfcc.predict(X2_test, batch_size=64).argmax(axis=1)
print("\n[CNN MFCC2D] Test accuracy:", (y2_pred == y2_test).mean())
print("[CNN MFCC2D] Macro-F1:", f1_score(y2_test, y2_pred, average="macro"))
print(classification_report(y2_test, y2_pred, target_names=le.classes_))

plot_history(history_cnn_mfcc, title_prefix="[CNN MFCC2D]")
plot_confusion(y2_test, y2_pred, labels=le.classes_, title="[CNN MFCC2D] Matrice de confusion (normalisée)")

model_cnn_mfcc.save("/content/drive/MyDrive/IA/Model Save/Model_MFCC_2D.keras")


Réseau de neurones convolutif (CNN, Log-Mel)

Une troisième variante utilise les log-Mel spectrogrammes comme entrée, offrant une représentation plus proche de la perception humaine du son.
Cette méthode conserve les avantages du CNN tout en exploitant la richesse fréquentielle du log-Mel.
Le modèle obtient environ 69 % d’accuracy test (macro-F1 ≈ 0.70).
Bien que légèrement inférieur au CNN-MFCC, il confirme la pertinence de cette représentation spectrale et ouvre la voie à des architectures hybrides plus puissantes.

In [ ]:
# ============================================================
# PIPELINE 3 — CNN sur Log-Mel
# ============================================================
Xl_list, yl_list = [], []
for fp, yi in tqdm(zip(label_df["filepath"].values, y_all_ref),
                   total=len(label_df), desc="Log-Mel (CNN)"):
    try:
        y_wav = load_fixed_mono(fp, sr=SR, target_dur=TARGET_DUR)
        feat  = logmel_2d(y_wav, sr=SR, n_fft=N_FFT, hop=HOP, n_mels=N_MELS, norm="per_band")
        feat  = feat[..., np.newaxis]  # (N_MELS, T, 1)
        Xl_list.append(feat)
        yl_list.append(yi)
    except Exception as e:
        print(f"Skip {fp} -> {e}")

X_logmel = np.stack(Xl_list)
y_logmel = np.array(yl_list, dtype=int)
print("LogMel — Features:", X_logmel.shape, "| Labels:", y_logmel.shape)

X_train_lm, X_val_lm, X_test_lm, y_train_lm, y_val_lm, y_test_lm = slice_split(X_logmel, y_logmel)
num_classes_cnn_lm = len(np.unique(y_logmel))

# Modèle CNN LogMel (même archi)
inp_lm = layers.Input(shape=(X_train_lm.shape[1], X_train_lm.shape[2], 1))
x = layers.Conv2D(32, (3,3), padding="same", activation="relu")(inp_lm)
x = layers.BatchNormalization()(x); x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(64, (3,3), padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x); x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(128, (3,3), padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x); x = layers.MaxPooling2D((2,2))(x)
x = layers.Dropout(0.4)(x); x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x); x = layers.Dropout(0.4)(x)
out_lm = layers.Dense(num_classes_cnn_lm, activation="softmax")(x)

model_cnn_logmel = keras.Model(inp_lm, out_lm)
model_cnn_logmel.compile(optimizer=keras.optimizers.Adam(1e-3),
                         loss="sparse_categorical_crossentropy",
                         metrics=["accuracy"])
cbs_cnn_lm = [
    callbacks.EarlyStopping(patience=6, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(patience=3, factor=0.5),
]
history_cnn_logmel = model_cnn_logmel.fit(
    X_train_lm, y_train_lm, validation_data=(X_val_lm, y_val_lm),
    epochs=40, batch_size=64, callbacks=cbs_cnn_lm, verbose=1
)

y_pred_lm = model_cnn_logmel.predict(X_test_lm, batch_size=64).argmax(axis=1)
print("\n[CNN LogMel] Test accuracy:", (y_pred_lm == y_test_lm).mean())
print("[CNN LogMel] Macro-F1:", f1_score(y_test_lm, y_pred_lm, average="macro"))
print(classification_report(y_test_lm, y_pred_lm, target_names=le.classes_))

plot_history(history_cnn_logmel, title_prefix="[CNN LogMel]")
plot_confusion(y_test_lm, y_pred_lm, labels=le.classes_, title="[CNN LogMel] Matrice de confusion (normalisée)")

model_cnn_logmel.save("/content/drive/MyDrive/IA/Model Save/Model_CNN_LogMel.keras")

Réseau CRNN (CNN + BiGRU, Log-Mel)

Le quatrième modèle combine des couches convolutives pour extraire des motifs locaux et un réseau récurrent bidirectionnel (BiGRU) pour modéliser la dynamique temporelle du chant.
Cette architecture dite CRNN permet de capturer à la fois les structures fréquentielles et les dépendances temporelles à long terme.
Elle atteint environ 78 % d’accuracy test et un macro-F1 ≈ 0.78, illustrant un net gain par rapport aux CNN purs.
Les classes bewickii et migratorius sont particulièrement bien reconnues (rappel ≈ 0.80).

In [ ]:
# ============================================================
# PIPELINE 4 — CRNN (CNN + BiGRU) sur Log-Mel
# ============================================================

Xcr_list, ycr_list = [], []
for fp, yi in tqdm(zip(label_df["filepath"].values, y_all_ref),
                   total=len(label_df), desc="Log-Mel (CRNN)"):
    try:
        y_wav = load_fixed_mono(fp, sr=SR, target_dur=TARGET_DUR)
        feat  = logmel_2d(y_wav, sr=SR, n_fft=N_FFT, hop=HOP, n_mels=N_MELS, norm="per_band")
        feat  = feat[..., np.newaxis]  # (F, T, 1)
        Xcr_list.append(feat)
        ycr_list.append(yi)
    except Exception as e:
        print(f"Skip {fp} -> {e}")

X_crnn = np.stack(Xcr_list)
y_crnn = np.array(ycr_list, dtype=int)
print("CRNN — Features:", X_crnn.shape, "| Labels:", y_crnn.shape)

X_train_c, X_val_c, X_test_c, y_train_c, y_val_c, y_test_c = slice_split(X_crnn, y_crnn)

# class_weight sur le TRAIN
classes_unique = np.unique(y_train_c)
cw = compute_class_weight(class_weight="balanced", classes=classes_unique, y=y_train_c)
class_weight = {int(c): float(w) for c, w in zip(classes_unique, cw)}
print("class_weight:", class_weight)

num_classes_crnn = len(np.unique(y_crnn))

# Modèle CRNN (CNN → BiGRU → GlobalAveragePooling1D → Dense)
def conv_block(x, filters, pool=(2,2), drop=0.0):
    x = layers.Conv2D(filters, (3,3), padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D(pool)(x)
    if drop and drop > 0:
        x = layers.Dropout(drop)(x)
    return x

inp = layers.Input(shape=(X_train_c.shape[1], X_train_c.shape[2], 1))  # (F, T, 1)
x = conv_block(inp,  32, pool=(2,2), drop=0.10)
x = conv_block(x,    64, pool=(2,2), drop=0.10)
x = conv_block(x,   128, pool=(2,1), drop=0.20)
# (F', T', C) -> (T', F', C) -> (T', F'*C)
x = layers.Permute((2,1,3))(x)
x = layers.TimeDistributed(layers.Flatten())(x)
# RNN bidirectionnel (GRU)
x = layers.Bidirectional(layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(x)
# Agrégation temporelle moyenne
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.4)(x)
out = layers.Dense(num_classes_crnn, activation="softmax")(x)

model_crnn = keras.Model(inp, out)
model_crnn.compile(optimizer=keras.optimizers.Adam(1e-3),
                   loss="sparse_categorical_crossentropy",
                   metrics=["accuracy"])

cbs_crnn = [
    callbacks.EarlyStopping(patience=6, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(patience=3, factor=0.5),
]

history_crnn = model_crnn.fit(
    X_train_c, y_train_c,
    validation_data=(X_val_c, y_val_c),
    epochs=40, batch_size=64, callbacks=cbs_crnn, verbose=1,
    class_weight=class_weight
)

# Scores
y_pred_c = model_crnn.predict(X_test_c, batch_size=64).argmax(axis=1)
print("\n[CRNN] Test accuracy:", (y_pred_c == y_test_c).mean())
print("[CRNN] Macro-F1:", f1_score(y_test_c, y_pred_c, average="macro"))
print(classification_report(y_test_c, y_pred_c, target_names=le.classes_))

plot_history(history_crnn, title_prefix="[CRNN Log-Mel]")
plot_confusion(y_test_c, y_pred_c, labels=le.classes_, title="[CRNN Log-Mel] Matrice de confusion (normalisée)")

model_crnn.save("/content/drive/MyDrive/IA/Model Save/Model_CRNN_LogMel.keras")


Réseau CRNN avec Attention (Log-Mel)

Le cinquième modèle enrichit le CRNN précédent par un mécanisme d’attention temporelle permettant au réseau de pondérer différemment chaque trame du signal selon sa pertinence.
Cette approche vise à concentrer l’attention du modèle sur les segments les plus informatifs du chant.
Les performances se stabilisent autour de 74 % d’accuracy test (macro-F1 ≈ 0.73), comparables au CNN 2D mais avec une meilleure robustesse aux variations temporelles.
Le modèle démontre une bonne capacité de généralisation et une interprétabilité accrue via la visualisation des poids d’attention.

In [ ]:
# ============================================================
# PIPELINE 5 — CRNN + Temporal Pooling Appris sur Log-Mel
# ============================================================

Xatt_list, yatt_list = [], []
for fp, yi in tqdm(zip(label_df["filepath"].values, y_all_ref),
                   total=len(label_df), desc="Log-Mel (CRNN+Attention)"):
    try:
        y_wav = load_fixed_mono(fp, sr=SR, target_dur=TARGET_DUR)
        feat  = logmel_2d(y_wav, sr=SR, n_fft=N_FFT, hop=HOP, n_mels=N_MELS, norm="per_band")
        feat  = feat[..., np.newaxis]
        Xatt_list.append(feat)
        yatt_list.append(yi)
    except Exception as e:
        print(f"Skip {fp} -> {e}")

X_att = np.stack(Xatt_list)
y_att = np.array(yatt_list, dtype=int)
print("CRNN+Att — Features:", X_att.shape, "| Labels:", y_att.shape)

X_train_a, X_val_a, X_test_a, y_train_a, y_val_a, y_test_a = slice_split(X_att, y_att)

classes_unique = np.unique(y_train_a)
cw = compute_class_weight(class_weight="balanced", classes=classes_unique, y=y_train_a)
class_weight = {int(c): float(w) for c, w in zip(classes_unique, cw)}
print("class_weight:", class_weight)

num_classes_att = len(np.unique(y_att))

# Modèle CRNN + Attention
def conv_block(x, filters, pool=(2,2), drop=0.0):
    x = layers.Conv2D(filters, (3,3), padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D(pool)(x)
    if drop and drop > 0:
        x = layers.Dropout(drop)(x)
    return x

inp = layers.Input(shape=(X_train_a.shape[1], X_train_a.shape[2], 1))
x = conv_block(inp,  32, pool=(2,2), drop=0.10)
x = conv_block(x,    64, pool=(2,2), drop=0.10)
x = conv_block(x,   128, pool=(2,1), drop=0.20)

# (F', T', C) -> (T', F', C) -> (T', F'*C)
x = layers.Permute((2,1,3))(x)
x = layers.TimeDistributed(layers.Flatten())(x)

# BiGRU temporel
x = layers.Bidirectional(layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(x)

# scores par trame
w = layers.Dense(1, activation="tanh")(x)         # (batch, T', 1)
a = layers.Softmax(axis=1, name="att_weights")(w) # (batch, T', 1)

x = layers.Multiply()([x, a])                     # (batch, T', feat)
x = layers.Lambda(lambda t: tf.reduce_sum(t, axis=1), name="att_pool")(x)  # (batch, feat)

x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.4)(x)
out = layers.Dense(num_classes_att, activation="softmax")(x)

model_crnn_att = keras.Model(inp, out)
model_crnn_att.compile(optimizer=keras.optimizers.Adam(1e-3),
                       loss="sparse_categorical_crossentropy",
                       metrics=["accuracy"])

cbs_crnn_att = [
    callbacks.EarlyStopping(patience=6, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(patience=3, factor=0.5),
]

history_crnn_att = model_crnn_att.fit(
    X_train_a, y_train_a,
    validation_data=(X_val_a, y_val_a),
    epochs=40, batch_size=64, callbacks=cbs_crnn_att, verbose=1,
    class_weight=class_weight
)

y_pred_a = model_crnn_att.predict(X_test_a, batch_size=64).argmax(axis=1)
print("\n[CRNN+Att] Test accuracy:", (y_pred_a == y_test_a).mean())
print("[CRNN+Att] Macro-F1:", f1_score(y_test_a, y_pred_a, average="macro"))
print(classification_report(y_test_a, y_pred_a, target_names=le.classes_))

plot_history(history_crnn_att, title_prefix="[CRNN+Attention Log-Mel]")
plot_confusion(y_test_a, y_pred_a, labels=le.classes_, title="[CRNN+Attention] Matrice de confusion (normalisée)")

model_crnn_att.save("/content/drive/MyDrive/IA/Model Save/Model_CRNN_Attention_LogMel.keras")


In [ ]:
TEST_WAV = "/content/drive/MyDrive/IA/Whistle/Sifflement.wav"
MODEL    = model_crnn

y = load_fixed_mono(TEST_WAV, sr=SR, target_dur=3.0)
LM = logmel_2d(y, sr=SR, n_fft=N_FFT, hop=HOP, n_mels=N_MELS, norm="per_band")  # (F, T)
X = LM[..., np.newaxis][np.newaxis, ...]  # (batch=1, F, T, C=1)

proba = MODEL.predict(X, verbose=0)[0]

top_idx = np.argsort(proba)[::-1]
for i in range(min(5, len(proba))):
    cls = le.classes_[top_idx[i]]
    sc  = float(proba[top_idx[i]])
    print(f"{i+1}. {cls:12s} — {sc:.3f}")


# Conclusion générale

Plusieurs informations se dégagent à la fois sur le choix du traitement audio que sur la structure du modèle de classification :

##1. Traitement du signal

Les résultats montrent que la qualité des features extraites joue un rôle déterminant dans la performance finale :

* Les MFCC vectorisés (MLP), bien que simples à mettre en œuvre, perdent la dimension temporelle du signal. Ils ne permettent donc pas au réseau d’exploiter la dynamique du chant.
→ Résultat : environ 46 % d’accuracy, limité par une représentation trop compacte.

* Les MFCC 2D (CNN) conservent la structure temps–fréquence. Le modèle parvient ainsi à détecter des motifs acoustiques typiques des espèces.
→ Amélioration nette : environ 74 % d’accuracy, soit un gain de +28 points par rapport au MLP.

* Les log-Mel spectrogrammes offrent une représentation plus perceptuellement cohérente (échelle quasi logarithmique comme l’oreille humaine).
→ Le score atteint environ 69 %, légèrement inférieur au CNN MFCC, mais plus robuste sur certaines classes minoritaires.


## 2. Architecture du modèle

Le passage d’un CNN pur à une architecture hybride CNN + RNN (CRNN) constitue une avancée :

* Le CRNN combine la détection locale des motifs fréquentiels (via les convolutions) et la modélisation temporelle à long terme (via GRU bidirectionnels).
→ Il atteint environ 78 % d’accuracy et macro-F1 ≈ 0.78, confirmant sa capacité à capturer à la fois la texture spectrale et l’évolution temporelle du chant.

* Le CRNN avec attention temporelle introduit une pondération adaptative sur les trames les plus pertinentes, améliorant la lisibilité et la stabilité du modèle.
→ Les performances se stabilisent autour de 74 %, légèrement inférieures au CRNN pur mais avec une meilleure interprétabilité grâce aux cartes d’attention.

##3. Analyse comparative

| Modèle   | Représentation | Type de réseau          | Accuracy test | Macro-F1 | Points forts                                   |
| -------- | -------------- | ----------------------- | ------------- | -------- | ---------------------------------------------- |
| MLP      | MFCC 1D        | Dense                   | 0.46          | 0.45     | Simple, rapide, baseline                       |
| CNN      | MFCC 2D        | Convolutif              | **0.74**      | **0.74** | Capture les motifs temps-fréquence             |
| CNN      | Log-Mel 2D     | Convolutif              | 0.69          | 0.70     | Représentation perceptuelle                    |
| CRNN     | Log-Mel 2D     | CNN + BiGRU             | **0.78**      | **0.78** | Meilleur compromis entre structure et séquence |
| CRNN+Att | Log-Mel 2D     | CNN + BiGRU + Attention | 0.74          | 0.73     | Interprétable, stable                          |


##4. Bilan final

En résumé :

* Le meilleur modèle global est le CRNN (CNN + BiGRU) entraîné sur les log-Mel spectrogrammes.

* Il tire parti des caractéristiques spectrales locales et des relations temporelles longues, ce qui est un très bon point pour la reconnaissance de chants d’oiseaux.

* L’ajout d’un mécanisme d’attention n’a pas amélioré la performance.

* Ainsi, le meilleur traitement reste le log-Mel 2D, couplé à une architecture hybride convolutive et récurrente, confirmant que l’intégration du temps dans l’apprentissage est cruciale pour la classification sonore.

